In [2]:
### Benjamin Tollison ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [5]:
# Newton method for Eccentric anomoly
# part a
e = 0.5
mean_anomoly = np.pi/4
f = lambda E: E - e*np.sin(E) - mean_anomoly
Delta_k = lambda E: f(E)/(1-e*np.cos(E))
step_size = lambda E: np.linalg.norm(Delta_k(E))
true_anomoly = lambda E: 2*np.arctan2(np.sqrt((1+e**2)/(1-e**2))*np.tan(E/2),1)
E_i = np.pi/4
tolerance = 1e-8
Newton_method_data = {'function error':[],'E_k':[],'step size':[],'f_k':[]}
while True:
  Newton_method_data['function error'].append(f(E_i))
  Newton_method_data['E_k'].append(E_i)
  Newton_method_data['step size'].append(Delta_k(E_i))
  Newton_method_data['f_k'].append(true_anomoly(E_i))
  if f(E_i) < tolerance and f(E_i) > -tolerance:
    break
  E_i = E_i - (f(E_i) / (1-e*np.cos(E_i)))
pd.DataFrame(Newton_method_data)

,function error,E_k,step size,f_k
0,-3.535534e-01,0.785398,-5.469182e-01,0.982115
1,6.106908e-02,1.332316,6.924819e-02,1.585435
2,1.157898e-03,1.263068,1.364556e-03,1.513537
3,4.435719e-07,1.261704,5.231400e-07,1.512107
4,6.517009e-14,1.261703,7.686036e-14,1.512107
